In [1426]:
import pandas as pd
import numpy as np
import pylab as pl
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing

In [1427]:
df_train = pd.read_csv("Comp_train.csv", delimiter=';', header=None)
df_train = df_train.sample(frac=1).reset_index(drop=True)
#перемешивает тренировочную выборку для нормальной кросс-валидации

In [1428]:
print(df_train.shape)

(300, 10)


In [1429]:
Y_train = df_train[9].values.astype(np.int)
X_train = df_train.drop([9], axis=1)
print(Y_train.shape)
print(X_train.shape)

for col in X_train.columns:
    s = X_train[col].value_counts()
    print(len(s))
# количество градаций в каждом признаке

(300,)
(300, 9)
33
4
4
14
2
2
4
4
2


In [1430]:
from copy import deepcopy

def dummy_encode_categorical_columns(df, categorical_columns):
    result_df = deepcopy(df)
    for column in categorical_columns:
        result_df = pd.concat([result_df, pd.get_dummies(result_df[column], 
                                                         prefix=column, prefix_sep=': ')], axis=1)
        del result_df[column]
    return result_df


In [1431]:
categorical_columns = (1,2,6,7) #в этих признаках по 4 градации
df_no_cat = dummy_encode_categorical_columns(X_train, categorical_columns)
X_train=df_no_cat.values.astype(np.float32)
print(X_train.shape)

(300, 21)


In [1432]:
def cross_validate(X, y, clf, splits=6, sel_feat=False, del_anom=False):
    X = preprocessing.normalize(X, axis=0, norm='max')
    scores = []
    kf = KFold(n_splits=splits)
    for train, test in kf.split(X):
        x_train, x_test, y_train, y_test = X[train], X[test], y[train], y[test]

        if sel_feat:
            estimator = DecisionTreeClassifier()
            selector = RFECV(estimator, step=1, n_jobs=-1)
            x_train = selector.fit_transform(x_train, y_train)
            x_test = x_test[:,selector.ranking_==1]
        
        if del_anom:
            lof = LocalOutlierFactor(n_neighbors=10, contamination=0.05)
            keep = lof.fit_predict(x_train)
            x_train = x_train[keep==1,:]
            y_train = y_train[keep==1]          

        p = clf.fit(x_train, y_train).predict(x_test)
        score = accuracy_score(y_test, p)
        scores.append(score)
    return scores


In [1433]:
clf = svm.SVC(C=3.5, kernel='sigmoid')

a = cross_validate(X_train, Y_train, clf, del_anom=True)

print(mean(a))

0.456666666667


In [1434]:
df_test = pd.read_csv("Comp_test.csv", delimiter=';', header=None)
df_test = df_test.drop(range(9,15), axis=1)
print(df_test.shape)

(700, 9)


In [1435]:
df_no_cat = dummy_encode_categorical_columns(df_test, categorical_columns)
df_test=df_no_cat.values.astype(np.float32)

df_test = preprocessing.normalize(df_test, axis=0, norm='max')

In [1436]:
a = clf.predict(df_test)
df = pd.DataFrame(a)
with open('result.csv', 'w') as f:
    f.write(df.to_csv(header=False, index=False))
    